# 营销活动优化


## 目的和先决条件

此 Jupyter Notebook 描述了银行和金融服务行业常见的营销活动优化问题。该问题使用 Gurobi Python API 构建，并使用 Gurobi Optimizer 解决。

在这个营销活动问题中，数学优化模型的关键参数是通过机器学习预测响应模型预估出来的。营销活动优化问题是指在满足各种业务约束的情况下，为使总预期利润最大化而向每个客户提供何种产品。

该建模示例处于初级阶段，我们假设你了解Python，并且具有一些有关构建数学优化模型的知识。希望你可以从示例中了解所有遗漏的概念。

**注意：** 你可以通过单击 [此处](https://github.com/arvinxx/gurobi-and-mathematical-modeling/archive/master.zip) 下载包含此示例和其他示例的代码。为了正确运行此 Jupyter Notebook，你必须具有 Gurobi 许可证。如果你没有，则可以**商业用户身份**申请 [试用许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/)，或以**学术用户身份**下载 [免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses)。

## 动机

银行和金融服务行业营销的主要目标是“在正确的时间向正确的客户提供正确的产品”。然而，实际上能够实现这一目标是一项复杂且具有挑战性的任务。让这变得特别困难的是，公司拥有多种产品，并在一组复杂的业务约束下运作。为了最大化市场投资回报和满足业务约束，选择向哪些客户提供哪些产品是非常复杂的。

假设有一家大型银行，该银行已努力成为一家以客户为中心的机构，而不是一家垂直产品驱动型公司。该银行的目标是“通过提供满足客户独特需求的解决方案，尽最大可能帮助他们在经济上有所改善”。此目标的直接结果是，营销活动是多个产品活动，而不是单个产品活动。

这将数据科学和营销活动的定位过程从相当简单的单个响应模型变为一个相当复杂的过程，包括选择哪个产品通过哪个渠道提供给哪个客户。


银行的营销团队习惯于将业务规则直接应用于目标客户。例如，他们仅根据产品差距或营销人员的业务直觉来瞄准客户。该银行的营销人员还应用了RFM类型分析，使用一般近因、频率、货币度量以及产品缺口来针对特定的目标客户。

营销团队目前广泛使用的方法是依赖预测响应模型来锁定客户。这些模型估计客户响应特定报价的可能性，并且可以显着提高对产品报价的响应率。然而，当一家公司有好几种产品要推销，在营销计划中要考虑其他业务约束时，仅仅知道客户对特定报价做出响应概率是不够的。

一般来说，营销团队还面临一个问题，就是要知道该向客户提供哪种产品，而不仅仅是该向哪个客户提供产品。在实践中，使用了许多特殊规则:

* 基于响应率或估计预期盈利能力指标的优先级规则

* 对可以销售的产品进行优先级排序的业务规则

* 为特定活动选择客户的产品响应模型。

有一种方法很容易实现，但可能不能产生最佳的客户联系方案，它依赖于预期报价获利能力的指标，以选择向客户提供哪些产品。然而，这种方法的一个缺点是它不能有效地处理客户联系方案上的复杂约束。

为了解决这个营销活动优化问题，M.D.Cohen[1] 提出了一种基于丰业银行数据的MIP方法。营销活动优化问题考虑了11种独特的报价：5种投资、3种贷款和3种日常银行报价。投资优惠包括担保投资证书（GICs）、共同基金、注册教育储蓄计划（RESP）和两个独特的折扣经纪服务。贷款优惠包括一项抵押贷款和两项信用卡贷款。

日常银行服务包括两项 Scotia 网上银行服务中的一项和存款账户获取服务。在这里，“活动”一词指的是一个大型的、积极主动的客户联系活动，包括11个不同的优惠;它可以被认为是十一个单一的产品活动，通常是在同一时间提供给一个不重叠的客户群。该活动的潜在目标市场包括大约250万客户。

在此Jupyter Notebook中，我们将使用此MIP方法来解决银行的营销活动优化问题。应该注意的是，几乎所有行业的任何公司都可以使用这种方法在其业务限制优化营销活动方案。

## 问题描述

银行的营销团队需要确定向每个客户提供什么样的产品，以使营销活动的投资回报最大化，同时考虑以下限制因素：

* 活动可用资金的限制。

* 活动中可以提供的最低产品数量的限制。

* 活动必须满足的投资回报率最低要求。

## 解决方法


数学优化（也称为数学编程）是一种声明性方法，其中建模者制定了一个优化问题，该问题捕获了复杂决策问题的关键特征。然后 Gurobi 使用最先进的数学和计算机科学技术来解决这一类数学优化问题。


数学优化模型具有五个组成部分：

* 数据集（Sets）
* 参数(Parameters)
* 决策变量(Decision variables)
* 约束（Constraints）
* 目标函数（Objective function(s)）


现在，我们针对此营销活动优化问题提出一种MIP方法。

Cohen[1]提出的MIP解决方案方法是对传统的短视方法（myopic approach）的改进，即选择对特定产品具有最大预期价值的客户，因为它从银行的角度产生了一个全局最优的解决方案，并允许有效地构建有关客户和业务部门的业务约束。这种方法可以考虑到有限的资源和其他业务限制。

我们假设对客户/要约的预期增量利润，成本和业务约束的估计可以用作营销活动优化方法的输入。优化阶段与这些输入的构造无关。

MIP方法涉及一个策略和操作问题。

对于策略问题，我们根据单个预期利润参数汇总客户。可以使用诸如预测响应模型之类的数据科学技术来确定估计的个人预期利润。关键思想是对估计的单个预期利润进行聚类，然后将聚类质心视为代表单个聚类中所有单个客户的数据的代表。这种聚合使问题可以表述为线性规划问题，因此该模型无需为单个客户分配报价，而是在考虑业务约束的同时，为每个产品报价在每个集群中确定要最大化营销活动投资回报率的比例。通常，集群中的客户数量将成千上万，这是策略问题的主要决策变量，因此，这些变量可以视为连续变量；因此，线性规划方法是合理的。

操作问题可以表述为MIP模型，其中估计的个人预期利润和战术模型的输出可以用作输入，以将*产品报价* 分配给每个集群的单个客户，从而使整个营销活动投资回报最大化。

## 策略模型制定


### 集合与索引

$k \in K$: 类的索引和集合。

$j \in J$: 产品的索引和集合。

### 参数
$\pi_{k,j}$: 将产品 $j \in J$ 提供给类 $k \in K$ 的客户时所获得的平均期望利润。

$\nu_{k,j}$: 将产品 $j \in J$ 提供给类 $k \in K$ 的客户时平均可变成本。
  
$N_{k}$: 类 $k \in K$ 中包含的客户数量。

$Q_{j}$: 产品 $j \in J$ 的最低要约数量。 

$R$: 企业最低回报率。此最低回报率用于计算营销活动的投资回报率。

$B$: 营销活动预算。


$M$: Big M 惩罚。此惩罚与预算修正相关，预算修正是满足其他业务限制所必需的。 

### 决策变量
$y_{k,j} \geq 0$: 类 $k \in K$  中的客户接受产品 $j \in J$ 要约的客户数量。

$z \geq 0$: 增加预算，以便有一个可行的营销活动。

### 目标函数

- **总利润**： 最大限度地提高营销活动的预期总利润，并对任何对预算的调整进行严厉惩罚。

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} - M \cdot z
\tag{0}
\end{equation}

### 约束条件

- **要约数量**： 每个集群中产品的最大要约数量受集群中客户数量的限制。

\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}

- **预算**： 营销活动预算约束要求活动的总成本应该低于活动预算。有可能需要增加预算以确保模型的可行性。因为为了使所有产品达到最低要约数量，可能需要增加预算。

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B + z
\tag{2}
\end{equation}

- **要约限制**： 每个产品的最低要约数量。

\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

- **ROI**： 最小 ROI 约束可确保总利润与成本的比率至少为 1 + 企业的最低预期回报率。

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}


## 操作模型制定

一旦找到了策略模型中所有 $j \in J$ 和 $k \in K$ 的最优值 $y_{k,j}$ ，我们就需要确定集群 $k$ 中的哪个个体客户得到哪个产品的要约。假设给定集群 $k \in K$ 中，产品 $j_1$ 和 $j_2$ 的要约分配为正，即 $y_{k,j_1} > 0$ 和$y_{k,j_2} > 0$。那么，集群 $k$ 中的顾客 $y_{k,j_1}$ 和 $y_{k,j_2}$ 必须分别是产品 $j_1 $和 $j_2$ 的要约。要做到这一点，最优的方法是使用单个客户的预期利润来解决分配问题，而不是使用集群的预期利润。

我们现在提供操作问题的表述。


### 集合与索引
$i \in I^{k}$: 集群 $k \in K$ 中客户的索引和集合。

$j \in J^{k}$: 集群 $k \in K$ 中提供给客户的产品索引和子集，其中 $J^{k} = \{ j \in J: y_{k,j} > 0 \}$ .

### 参数

$r_{k,i,j}$: 从客户 $i \in I^{k}$ 的产品要约 $j \in J^{k}$ 的预期个人利润。 

$Y_{k,j} = \lfloor y_{k,j} \rfloor $: 在集群 $k$ 中产品 $j \in J^{k}$ 提供要约的客户数量
.

### 决策变量
$x_{k,i,j} \in \{0,1 \}$: 如果产品 $j \in J^{k}$  给客户 $i \in I^{k}$ 提供了要约，此变量等于1, 否则为 0。



### 目标函数
- **总利润**： 最大化总个人利润。


\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}


### 约束

- **产品要约**： 将产品的要约分配给每个集群的客户。

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}


- **要约限制**： 最多可以向集群的一个客户提供一个产品。


\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

- **布尔限制**： 是否向集群 $k$ 的客户提供产品要约。


\begin{equation}
x_{k,i,j} \in \{0,1 \} \quad \forall i \in I^{k},  j \in J^{k}, k \in K
\tag{3}
\end{equation}


## 问题实例

我们考虑两个产品、十个客户和两个客户群。公司的最低回报率预期是百分之二十。

### 策略问题数据

下表定义了提供产品时每个集群中平均客户的预期利润。

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1 | $\$2,000$ | $\$1,000$ |
| cluster 2 | $\$3,000$ | $\$2,000$ |

下表确定了向集群中的普通客户提供产品的预期成本。

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1 | $\$200$ | $\$100$ |
| cluster 2 | $\$300$ | $\$200$ |

这个营销活动的预算是$\$200$。

下表给出了每个集群中的客户数量。

| <i></i> | Num. Customers | 
| --- | --- |
| cluster 1 | 5 |
| cluster 2 | 5 | 

下表列出每种产品的最低要约数量。

| <i></i> | Min Offers | 
| --- | --- |
| product 1 | 2 |
| product 2 | 2 | 

### 操作问题数据

下表显示了向每个集群中每个客户提供产品的预期利润。

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1, customer 1 | $\$2,050$ | $\$1,050$ |
| cluster 1, customer 2 | $\$1,950$ | $\$950$ |
| cluster 1, customer 3 | $\$2,000$ | $\$1,000$ |
| cluster 1, customer 4 | $\$2,100$ | $\$1,100$ |
| cluster 1, customer 5 | $\$1,900$ | $\$900$ |
| cluster 2, customer 6 | $\$3,000$ | $\$2,000$ |
| cluster 2, customer 7 | $\$2,900$ | $\$1,900$ |
| cluster 2, customer 8 | $\$3,050$ | $\$2,050$ |
| cluster 2, customer 9 | $\$3,100$ | $\$2,100$ |
| cluster 2, customer 10 | $\$2,950$ | $\$1,950$ |

下表显示了向集群中的客户提供产品的成本。

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1, customer 1 | $\$205$ | $\$105$ |
| cluster 1, customer 2 | $\$195$ | $\$95$ |
| cluster 1, customer 3 | $\$200$ | $\$100$ |
| cluster 1, customer 4 | $\$210$ | $\$110$ |
| cluster 1, customer 5 | $\$190$ | $\$90$ |
| cluster 2, customer 6 | $\$300$ | $\$200$ |
| cluster 2, customer 7 | $\$290$ | $\$190$ |
| cluster 2, customer 8 | $\$305$ | $\$205$ |
| cluster 2, customer 9 | $\$310$ | $\$210$ |
| cluster 2, customer 10 | $\$295$ | $\$195$ |

## Python 实现

我们现在导入Gurobi Python模块。然后用给定的数据初始化数据结构。

In [1]:
import gurobipy as gp
from gurobipy import GRB

# 官方测试版本 Gurobi v9.0.0 & Python 3.7.0

# 译者测试版本 Gurobi v9.1.0 & Python 3.8.6

### 集合

products = ['p1', 'p2']
clusters = ['k1', 'k2']

### 预期利润

下表显示了每个集群中提供要约的客户的预期利润。

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1 | $\$2,000$ | $\$1,000$ |
| cluster 2 | $\$3,000$ | $\$2,000$ |

In [2]:
### 参数

# 预期利润
cp, expected_profit = gp.multidict({
    ('k1', 'p1'): 2000,
    ('k1', 'p2'): 1000,
    ('k2', 'p1'): 3000,
    ('k2', 'p2'): 2000
})


### 预期成本

下表显示了向集群中的客户提供产品的预期成本。

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1 | $\$200$ | $\$100$ |
| cluster 2 | $\$300$ | $\$200$ |

In [3]:
# 预期成本

cp, expected_cost = gp.multidict({
    ('k1', 'p1'): 200,
    ('k1', 'p2'): 100,
    ('k2', 'p1'): 300,
    ('k2', 'p2'): 200
})

### 客户数量

每个集群中的客户数量如下表所示。

| <i></i> | Num. Customers | 
| --- | --- |
| cluster 1 | 5 |
| cluster 2 | 5 | 

In [4]:
# 每个集群中的客户数量

clusters, number_customers = gp.multidict({
    ('k1'): 5,
    ('k2'): 5
})

### 最低要约

下表列出每种产品的最低要约数量。

| <i></i> | 最低要约 | 
| --- | --- |
| product 1 | 2 |
| product 2 | 2 | 

In [31]:
#每个产品的最低要约数量

products, min_offers = gp.multidict({
    ('p1'): 2,
    ('p2'): 2
})

### 标量

公司的最低回报率是百分之二十 ($R = 0.20$).

可用于市场营销活动的预算是 $\$200$.


In [6]:
# 标量

R = 0.20

# 预算紧张
budget = 200 

## 策略模型制定

 

### 决策变量
$y_{k,j} \geq 0$: 类 $k \in K$  中的客户接受产品 $j \in J$ 要约的客户数量。

$z \geq 0$: 增加预算，以便有一个可行的营销活动。


In [32]:
# 声明并初始化模型
mt = gp.Model('Tactical')

### 决策变量

# 集群式分配产品要约给客户。

y = mt.addVars(cp, name="allocate")

# 修正预算

z = mt.addVar(name="budget_correction")

### 约束


- **要约数量**： 每个集群中产品的最大要约数量受集群中客户数量的限制。

\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}


Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$N_{k}$: Number of customers in cluster $k \in K$.

In [34]:
### 约束

# Constraint on number of offers at each cluster

maxOffers_cons = mt.addConstrs((y.sum(k,'*') <= number_customers[k]  for k in clusters), name='maxOffers')


### Constraints

- **Budget**. The marketing campaign budget constraint enforces that the total cost of the campaign should be less than the budget campaign. There is the possibility of increasing the budget to ensure the feasibility of the model, the minimum number of offers for all the product may require this increase in the budget.

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B + z
\tag{2}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$z \geq 0$: Increase in budget in order to have a feasible campaign.

$\nu_{k,j}$: Average variable cost associated with the offer of product  $j \in J$ to an average customer of cluster $k \in K$.

$B$: Marketing campaign budget.

In [35]:
# Budget constraint

budget_con = mt.addConstr((y.prod(expected_cost) - z <= budget), name='budget')


### Constraints

- **Offers limit**. Minimum number of offers of each product.

\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$Q_{j}$: Minimum number of offers of product $j \in J$.

In [36]:
# Constraints on min number of offers of each product

minOffers_cons = mt.addConstrs( (y.sum('*',j) >= min_offers[j] for j in products), name='min_offers')


### Constraints


- **ROI**. The minimum ROI constraint ensures that the ratio of total profits over cost is at least  one plus the corporate hurdle rate.

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$\pi_{k,j}$: Expected profit to the bank from the offer of product $j \in J$ to an average customer of cluster $k \in K$.

$\nu_{k,j}$: Average variable cost associated with the offer of product  $j \in J$ to an average customer of cluster $k \in K$.

$R$: Corporate hurdle rate.

In [37]:
# Constraint to ensure minimum ROI

ROI_con = mt.addConstr((y.prod(expected_profit) - (1 + R)*y.prod(expected_cost) >= 0), name='ROI')

### Objective Function
- **Total profit**. Maximize total expected profit from marketing campaign and heavily penalize any correction to the budget.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} - M \cdot z
\tag{0}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$z \geq 0$: Increase in budget in order to have a feasible campaign.

$\pi_{k,j}$: Expected profit to the bank from the offer of product $j \in J$ to an average customer of cluster $k \in K$.

**Note:** The value of $M$ should be higher than any of the expected profits to ensure that the budget is increased only when the model is infeasible if this parameter is not increased.

In [38]:
### Objective function

# Maximize total expected profit

M = 10000

mt.setObjective(y.prod(expected_profit) -M*z, GRB.MAXIMIZE)

In [39]:
# Verify model formulation

mt.write('tactical.lp')

In [40]:
# Run optimization engine

mt.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 5 columns and 21 nonzeros
Model fingerprint: 0x1bbca55e
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+03, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+04   8.787500e+01   0.000000e+00      0s
       4   -3.9940000e+06   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective -3.994000000e+06


In [41]:
### Output Reports

# Optimal allocation of product offers to clusters

total_expected_profit = 0
total_expected_cost = 0

print("\nOptimal allocation of product offers to clusters.")
print("___________________________________________________")
for k,p in cp:
    if y[k,p].x > 1e-6:
        #print(y[k,p].varName, y[k,p].x)
        print(f"The number of customers in cluster {k} that gets an offer of product {p} is: {y[k,p].x}")
        total_expected_profit += expected_profit[k,p]*y[k,p].x
        total_expected_cost += expected_cost[k,p]*y[k,p].x

increased_budget = '${:,.2f}'.format(z.x)
print(f"\nThe increase correction in the campaign budget is {increased_budget}.")

# Financial reports

optimal_ROI = round(100*total_expected_profit/total_expected_cost,2)
min_ROI = round(100*(1+R),2)

money_expected_profit = '${:,.2f}'.format(total_expected_profit)
money_expected_cost = '${:,.2f}'.format(total_expected_cost)
money_budget = '${:,.2f}'.format(budget)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total expected profit is {money_expected_profit}.")
print(f"Optimal total expected cost is {money_expected_cost} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {optimal_ROI}% with a minimum ROI of  {min_ROI}%.")


Optimal allocation of product offers to clusters.
___________________________________________________
The number of customers in cluster k1 that gets an offer of product p1 is: 2.0
The number of customers in cluster k1 that gets an offer of product p2 is: 2.0

The increase correction in the campaign budget is $400.00.

Financial reports.
___________________________________________________
Optimal total expected profit is $6,000.00.
Optimal total expected cost is $600.00 with a budget of $200.00 and an extra amount of $400.00.
Optimal ROI is 1000.0% with a minimum ROI of  120.0%.


## Analysis

The cost of allocating products to clusters required an increase in the budget of $\$400$. The total expected profit is 
$\$6,000$. The total expected cost is $\$600$ which is equal to the original budget of $\$200$ plus the increase of $\$400$. The expected ROI is 1,000 % which is much higher than the minimum ROI required.

## Operational Model Formulation

### Customer expected profit

In [42]:
### Sets

customers = ['c1', 'c2','c3','c4','c5','c6','c7','c8','c9','c10']

### Parameters

# Expected profit from a product offering for each customer in each cluster
ccp, customer_profit = gp.multidict({
    ('k1', 'c1', 'p1'): 2050,
    ('k1', 'c1', 'p2'): 1050,
    ('k1', 'c2', 'p1'): 1950,
    ('k1', 'c2', 'p2'): 950,
    ('k1', 'c3', 'p1'): 2000,
    ('k1', 'c3', 'p2'): 1000,
    ('k1', 'c4', 'p1'): 2100,
    ('k1', 'c4', 'p2'): 1100,
    ('k1', 'c5', 'p1'): 1900,
    ('k1', 'c5', 'p2'): 900,
    ('k2', 'c6', 'p1'): 3000,
    ('k2', 'c6', 'p2'): 2000,
    ('k2', 'c7', 'p1'): 2900,
    ('k2', 'c7', 'p2'): 1900,
    ('k2', 'c8', 'p1'): 3050,
    ('k2', 'c8','p2'): 2050,
    ('k2', 'c9', 'p1'): 3100,
    ('k2', 'c9', 'p2'): 3100,
    ('k2', 'c10', 'p1'): 2950,
    ('k2', 'c10', 'p2'): 2950   
})

### Customer offering cost

In [43]:
# Customer cost of offering a product at a cluster

ccp, customer_cost = gp.multidict({
    ('k1', 'c1', 'p1'): 205,
    ('k1', 'c1', 'p2'): 105,
    ('k1', 'c2', 'p1'): 195,
    ('k1', 'c2', 'p2'): 95,
    ('k1', 'c3', 'p1'): 200,
    ('k1', 'c3', 'p2'): 100,
    ('k1', 'c4', 'p1'): 210,
    ('k1', 'c4', 'p2'): 110,
    ('k1', 'c5', 'p1'): 190,
    ('k1', 'c5', 'p2'): 90,
    ('k2', 'c6', 'p1'): 300,
    ('k2', 'c6', 'p2'): 200,
    ('k2', 'c7', 'p1'): 290,
    ('k2', 'c7', 'p2'): 190,
    ('k2', 'c8', 'p1'): 305,
    ('k2', 'c8','p2'): 205,
    ('k2', 'c9', 'p1'): 310,
    ('k2', 'c9', 'p2'): 310,
    ('k2', 'c10', 'p1'): 295,
    ('k2', 'c10', 'p2'): 295   
})

## Operational Model Formulation


### Decision Variables
$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

In [44]:
# Declare and initialize model
mo = gp.Model('Operational')

### Decision variables

x = mo.addVars(ccp, vtype=GRB.BINARY, name="assign")

### Constraints

- **Product offers**. Allocate offers of a product to customers of each cluster.

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

$Y_{k,j} = \lfloor y_{k,j} \rfloor $: Number of customers in cluster k that will get an offer of product $j \in J^{k}$.




In [45]:
# Product offers constraint

productOffers = {}

for k in clusters:
    for j in products:
            productOffers[k,j] = mo.addConstr(gp.quicksum(x[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                              int(y[k,j].x), name='prodOffers_' + str(k) + ',' + str(j) )


### Constraints


- **Offers limit**. At most one product may be offered to a customer of a cluster.

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

In [46]:
# limit on the number of offers to each customer in a cluster.

ki = [('k1', 'c1'), 
      ('k1', 'c2'), 
      ('k1', 'c3'),
      ('k1', 'c4'), 
      ('k1', 'c5'), 
      ('k2', 'c6'), 
      ('k2', 'c7'), 
      ('k2', 'c8'), 
      ('k2', 'c9'), 
      ('k2', 'c10')]

customerOffers = {}

for k,i in ki:
    customerOffers[k,i] = mo.addConstr(gp.quicksum(x[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1, 
                                          name ='custOffers_' + str(k) + ',' + str(i) )

### Objective Function

- **Total profit**. Maximize total individual expected profit.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

$r_{k,i,j}$: Expected individual profit of customer $i \in I^{k}$ from  offer of product $j \in J^{k}$. 



In [47]:
### Objective function

# Maximoze total profit

mo.setObjective(x.prod(customer_profit), GRB.MAXIMIZE)

In [48]:
# Verify model formulation

mo.write('operational.lp')

# Run optimization engine

mo.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 14 rows, 20 columns and 40 nonzeros
Model fingerprint: 0xe1e9d99f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 6050.0000000
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 6.100000e+03, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6100.0000000 6100.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.01 seconds
Thread c

In [49]:
### Output Reports

# Optimal assignment of product offers to customers

total_customer_profit = 0
total_customer_cost = 0

kvalue = None
first = True
num_assignments = 0

print("\nOptimal assignment of product offers to customers.")
print("___________________________________________________")
for k,i,j in ccp:
    if k != kvalue:
        prevk = kvalue
        kvalue = k
        if not first:
            print("___________________________________________________")
            print(f"Number of assignments in cluster {prevk} is {num_assignments}")
            print("___________________________________________________")
            num_assignments = 0
        if first:
            first = False
    if x[k,i,j].x > 0.5:
        #print(x[k,i,j].varName, x[k,i,j].x)
        profit = '${:,.2f}'.format(customer_profit[k,i,j])
        cost = '${:,.2f}'.format(customer_cost[k,i,j])
        print(f"Customer {i} in cluster {k} gets an offer of product {j}:")
        print(f"The expected profit is {profit} at a cost of {cost}")
        total_customer_profit += customer_profit[k,i,j]*x[k,i,j].x
        total_customer_cost += customer_cost[k,i,j]*x[k,i,j].x
        num_assignments += 1
print("___________________________________________________")
print(f"Number of assignments in cluster {kvalue} is {num_assignments}")
print("___________________________________________________\n")
        
# Financial reports

customers_ROI = round(100*total_customer_profit/total_customer_cost,2)

money_customers_profit = '${:,.2f}'.format(total_customer_profit)
money_customers_cost = '${:,.2f}'.format(total_customer_cost)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total customers profit is {money_customers_profit}.")
print(f"Optimal total customers cost is {money_customers_cost} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {customers_ROI}% with a minimum ROI of  {min_ROI}%.")
        



Optimal assignment of product offers to customers.
___________________________________________________
Customer c1 in cluster k1 gets an offer of product p2:
The expected profit is $1,050.00 at a cost of $105.00
Customer c2 in cluster k1 gets an offer of product p2:
The expected profit is $950.00 at a cost of $95.00
Customer c3 in cluster k1 gets an offer of product p1:
The expected profit is $2,000.00 at a cost of $200.00
Customer c4 in cluster k1 gets an offer of product p1:
The expected profit is $2,100.00 at a cost of $210.00
___________________________________________________
Number of assignments in cluster k1 is 4
___________________________________________________
___________________________________________________
Number of assignments in cluster k2 is 0
___________________________________________________


Financial reports.
___________________________________________________
Optimal total customers profit is $6,100.00.
Optimal total customers cost is $610.00 with a budget o

## Analysis
Each customer got, at most, one product offer.  Product p2 is offered to customers c1 and c2, and product p1 is offered to customers c3 and c4. Products p1 and p2 are offerred to at least two customers -this is a constraint from the tactical model. Observe that to ensure these hard business constraints, the budget needs to be increased by $\$400$

The cost of assigning  products to customers is $\$610$, which slightly violates the total budget available of $\$600$. The total customers profit is $\$6,100$. The  ROI is 1,000 %, which is much higher than the minimum ROI required.

If the total available budget needs to be enforced, the following constraint can be added to the operational model:

- **Budget**. Enforce budget constraint.

\begin{equation}
\sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} c_{k,i,j} \cdot x_{k,i,j} \leq B'
\tag{4}
\end{equation}

The new budget is the original budget plus the correction, that is  $B' = B + z$

## Scenario 1
Enforce total budget available constraint. In this case, the operational model is:

### Objective function

- **Total profit**. Maximize total individual expected profit.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}

### Constraints

- **Product offers**. Allocate offers of a product to customers of each cluster.

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}

- **Offers limit**. At most one product may be offered to a customer in each cluster.

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

- **Budget**. Enforce budget constraint.

\begin{equation}
\sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} c_{k,i,j} \cdot x_{k,i,j} \leq B'
\tag{3}
\end{equation}

In [50]:
### Operational model enforcing constraint for total budget available 

# Declare and initialize model
mob = gp.Model('OperationalB')

### Decision variables

xb = mob.addVars(ccp, vtype=GRB.BINARY, name="assign")

In [51]:
# Product offers constraint

productOffersb = {}

for k in clusters:
    for j in products:
            productOffersb[k,j] = mob.addConstr(gp.quicksum(xb[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                              int(y[k,j].x), name='prodOffersb_' + str(k) + ',' + str(j) )

In [52]:
# limit on the number of offers to each customer in a cluster.

customerOffersb = {}

for k,i in ki:
    customerOffersb[k,i] = mob.addConstr(gp.quicksum(xb[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1, 
                                          name ='custOffersb_' + str(k) + ',' + str(i) )

In [53]:
# budget constraint

# New budget
new_budget = budget + z.x

totBudget = mob.addConstr(xb.prod(customer_cost) <= new_budget, name='total_budget')

In [54]:
### Objective function

# Maximize total profit

mob.setObjective(xb.prod(customer_profit), GRB.MAXIMIZE)

In [55]:
# Verify model formulation

mob.write('operationalB.lp')

# Run optimization engine

mob.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 15 rows, 20 columns and 60 nonzeros
Model fingerprint: 0x96da858f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [9e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 5950.0000000
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 8 rows, 10 columns, 28 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 6.000000e+03, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6000.0000000 6000.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.01 seconds
Thread c

In [56]:
### Output Reports

# Optimal assignment of product offers to customers

total_customer_profitb = 0
total_customer_costb = 0

kvalueb = None
firstb = True
num_assignmentsb = 0

print("\nOptimal assignment of product offers to customers.")
print("___________________________________________________")
for k,i,j in ccp:
    if k != kvalueb:
        prevkb = kvalueb
        kvalueb = k
        if not firstb:
            print("___________________________________________________")
            print(f"Number of assignments in cluster {prevkb} is {num_assignmentsb}")
            print("___________________________________________________")
            num_assignmentsb = 0
        if firstb:
            firstb = False
    if xb[k,i,j].x > 0.5:
        #print(x[k,i,j].varName, x[k,i,j].x)
        profitb = '${:,.2f}'.format(customer_profit[k,i,j])
        costb = '${:,.2f}'.format(customer_cost[k,i,j])
        print(f"Customer {i} in cluster {k} gets an offer of product {j}:")
        print(f"The expected profit is {profitb} at a cost of {costb}")
        total_customer_profitb += customer_profit[k,i,j]*xb[k,i,j].x
        total_customer_costb += customer_cost[k,i,j]*xb[k,i,j].x
        num_assignmentsb += 1
print("___________________________________________________")
print(f"Number of assignments in cluster {kvalueb} is {num_assignmentsb}")
print("___________________________________________________\n")
        
# Financial reports

customers_ROIb = round(100*total_customer_profitb/total_customer_costb,2)

money_customers_profitb = '${:,.2f}'.format(total_customer_profitb)
money_customers_costb = '${:,.2f}'.format(total_customer_costb)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total customers profit is {money_customers_profitb}.")
print(f"Optimal total customers cost is {money_customers_costb} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {customers_ROIb}% with a minimum ROI of  {min_ROI}%.")


Optimal assignment of product offers to customers.
___________________________________________________
Customer c1 in cluster k1 gets an offer of product p2:
The expected profit is $1,050.00 at a cost of $105.00
Customer c2 in cluster k1 gets an offer of product p1:
The expected profit is $1,950.00 at a cost of $195.00
Customer c4 in cluster k1 gets an offer of product p1:
The expected profit is $2,100.00 at a cost of $210.00
Customer c5 in cluster k1 gets an offer of product p2:
The expected profit is $900.00 at a cost of $90.00
___________________________________________________
Number of assignments in cluster k1 is 4
___________________________________________________
___________________________________________________
Number of assignments in cluster k2 is 0
___________________________________________________


Financial reports.
___________________________________________________
Optimal total customers profit is $6,000.00.
Optimal total customers cost is $600.00 with a budget o

## Analysis
Each customer got, at most, one product offer. Products p1 and p2 are offerred to at least two customers. The cost of assigning  products to customers is $\$600$, which is equal to the total budget available. The total customers profit is $\$6,000$. The  ROI is 1,000 %, which is much higher than the minimum ROI required.

In this scenario, we enforce the total available budget constraint and we get a different distribution for the assignment. 
Product p1 is offered to customers c2 and c4, and product p2 is offered to customers c1 and c5. 

##  Conclusion

In this Jupyter Notebook, we discussed the importance of marketing campaigns for the banking industry. We discussed that machine learning predictive response models can be used to provide the input data of a marketing campaign optimization problem. We showed how the marketing campaign optimization problem can be decomposed into a tactical problem and an operational problem.

The tactical problem is formulated as a linear programming problem where we aggregate data generated by the machine learning predictive response models.

The solution of the tactical problem determines what products to offer to customers in clusters while maximizing the marketing campaign expected profits and considering the following constraints: 

* limits on funding available for the campaign.
* restrictions on the minimum number of product offers that can be made in a campaign.
* campaign return-on-investment hurdle rates that must be met.

The operational problem is formulated as a MIP model, where the expected customer  profits and the output of the tactical model can be used as inputs to assign products offers to individual customers in such way that the total customers profit is maximized. We considered two cases for the operational problem. In the first case, the total available budget determined by tactical problem is not enforced. This means that the optimal solution of this problem might have slight violations of the total available budget. In the second case, we enforce a total available budget.


##  参考

[1] M. D. Cohen. *Exploiting response models—optimizing cross-sell and up-sell opportunities in banking.* Information Systems. Vol. 29. issue 4, June 2004, Pages 327-341


Copyright © 2020 Gurobi Optimization, LLC


